Jupyter Notebook analogous to nb_file.ipynb, but designed for SPARQL endpoints instead of local files.

It queries the endpoint via SPARQLWrapper, performs metric computations,

and exports the computed metric values to a CSV file for Streamlit integration.

In [ ]:
# parameters (will get overwritten)
source_path = "demo.ttl"       
metric_key  = "paths_depth"     
output_csv  = "metrics.csv"

In [ ]:
import pandas as pd, scrapbook as sb
from graph_loader import load_graph_from_file
import my_metrics as mm

# Load RDF graph data from the specified path.
g = load_graph_from_file(source_path)

# Mapping: Dictionary that associates metric keys (strings) to specific functions in the `my_metrics` module.
# Each metric function is expected to take the loaded RDF graph (`g`) as input and return a pandas DataFrame with results.
DISPATCH = {
    "paths_depth":   mm.paths_depth_local,
    "ont_tangledness":   mm.ont_tangledness_local,
    "degree_variance": mm.degree_variance_local,
    "primitives":    mm.primitives_local,
    "depth_of_inheritance_tree": mm.depth_of_inh_tree_local,
    "tbox":          mm.tbox_local,
    "abox":          mm.abox_local,
    "cohesion":      mm.cohesion_local,
}

# Retrieve the function to compute the desired metric using `metric_key` provided by the user.
function = DISPATCH.get(metric_key)
# If the metric is not found in the DISPATCH dictionary, an assertion error is raised.
assert function, f"Unknown metric: {metric_key}"

# Apply the retrieved function to the loaded RDF graph (`g`), resulting in a DataFrame of metric results.
df = function(g)

# index of metric won't be stored# Save the calculated metric results to a CSV file.
# `index=False` ensures that the DataFrame index is not included in the CSV file.
df.to_csv(output_csv, index=False)

# Displaying calculated metrics in streamlit environment.
sb.glue("metrics_table", df)